## Imports

https://tech.aaronteoh.com/creating-destinations-graph-wikivoyage-text/#:~:text=1.,(decompressor.decompress(data))

In [70]:
import os

import pandas as pd
import bz2
from bz2 import BZ2Decompressor
import mwxml

## Paths

In [37]:
PROJECT_DIR = os.path.dirname(os.getcwd())
DATA_DIR = "data/"
DATASET_PAGES_ARTICLES = "enwikivoyage-latest-pages-articles.xml"
DATASET_STUB_ARTICLES = "enwikivoyage-latest-stub-articles.xml"
DATASET_PAGES_ARTICLES_BZ2 = "enwikivoyage-latest-pages-articles-multistream.xml.bz2"

## EDA INIT

In [73]:
import os, urllib, json
from pprint import pprint
 
url = 'https://dumps.wikimedia.org/enwikivoyage/latest/enwikivoyage-latest-pages-articles.xml.bz2'
if not os.path.isfile(url.split('/')[-1]):
    #this step takes some time
    urllib.request.urlretrieve (url, url.split('/')[-1])

In [74]:
decompressed = ('.').join(url.split('/')[-1].split('.')[:-1])

In [75]:
from bz2 import BZ2Decompressor
 
if not os.path.isfile(decompressed):
    with open(decompressed, 'wb') as new_file, open(url.split('/')[-1], 'rb') as file:
        decompressor = BZ2Decompressor()
        for data in iter(lambda : file.read(100 * 1024), b''):
            new_file.write(decompressor.decompress(data))

In [76]:
if not os.path.isfile('wikivoyage_latest_articles_text.json'):
    import sys
    !{sys.executable} -m pip install xmltodict
    # https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
 
    import xmltodict
 
    #this step takes some time
    with open('enwikivoyage-latest-pages-articles.xml') as fd:
        doc = xmltodict.parse(fd.read())
 
    data = doc['mediawiki']['page']
    print('To process %s records' %len(data))
    del doc

To process 78358 records


In [78]:
data[0]

{'title': '1770',
 'ns': '0',
 'id': '1',
 'redirect': {'@title': 'Town of 1770'},
 'revision': {'id': '1',
  'timestamp': '2009-03-02T02:54:37Z',
  'contributor': {'username': 'Inas', 'id': '1816'},
  'comment': 'otherwise a number just looks weird.. (Import from wikitravel.org/en)',
  'origin': '1',
  'model': 'wikitext',
  'format': 'text/x-wiki',
  'text': {'@bytes': '26',
   '@sha1': 'jjresq501njc7hkah05ux8kk4t1y605',
   '@xml:space': 'preserve',
   '#text': '#REDIRECT [[Town of 1770]]'},
  'sha1': 'jjresq501njc7hkah05ux8kk4t1y605'}}

In [79]:
data[1]

{'title': '1liner',
 'ns': '0',
 'id': '2',
 'redirect': {'@title': 'Wikivoyage:One-liner listings'},
 'revision': {'id': '2',
  'timestamp': '2008-07-28T01:50:03Z',
  'contributor': {'username': 'Peterfitzgerald', 'id': '1768'},
  'minor': None,
  'comment': '[[WT:SHORT|shortcut]] to [[Project:One-liner listings]] (I can never remember this one) (Import from wikitravel.org/en)',
  'origin': '2',
  'model': 'wikitext',
  'format': 'text/x-wiki',
  'text': {'@bytes': '40',
   '@sha1': 'ixx0jbrmjhbgem6vavjxyy2w64t8vyu',
   '@xml:space': 'preserve',
   '#text': '#REDIRECT [[Project:One-liner listings]]'},
  'sha1': 'ixx0jbrmjhbgem6vavjxyy2w64t8vyu'}}

In [81]:
if not os.path.isfile('wikivoyage_latest_articles_text.json'):
    from collections import defaultdict
    completed =0
    articles = defaultdict(list)
     
    for item in data:
        if 'redirect' not in item:
            try:
                articles[item['title']].append(item['revision']['text']['#text'])
            except KeyError:
                continue
 
        completed+=1
        if completed%10000==0 or completed==len(data):
            print('Completed %s' %completed)
 
    print('Found %s articles' %len(articles))
    for article, text in articles.items():
        articles[article] = "".join(text)
 
    with open('wikivoyage_latest_articles_text.json', 'w') as f:
        json.dump(articles, f)
 
    del articles
    del data

Completed 10000
Completed 20000
Completed 30000
Completed 40000
Completed 50000
Completed 60000
Completed 70000
Found 43750 articles


## EDA

```
<title>: The title of the page.
<id>: The unique ID of the page.
<revision>: Contains details about a specific revision of the page, including:
<id>: Revision ID.
<timestamp>: When the revision was made.
<contributor>: Information about the user who made the revision.
<text>: The actual content of the page in wikitext format.
```

In [38]:
data_fpath = os.path.join(
    DATA_DIR,
    DATASET_PAGES_ARTICLES_BZ2,
)

In [52]:
f = bz2.open(data_fpath, "rb")
dump = mwxml.Dump.from_file(f)

In [53]:
?dump

Type:        Dump
String form: <mwxml.iteration.dump.Dump object at 0x168bddb00>
File:        ~/miniconda3/envs/llm_dev/lib/python3.12/site-packages/mwxml/iteration/dump.py
Docstring:  
XML Dump Iterator. Dump file meta data and a
:class:`~mwxml.iteration.page.Page` iterator.  Instances of this class can
be called as an iterator directly.  Usually, you'll want to construct this
class using :func:`~mwxml.iteration.dump.Dump.from_file`.

:Parameters:
    site_info : :class:`~mwxml.iteration.site_info.SiteInfo`
        The data from the <siteinfo> block
    pages : `iterable`
        An `iterable` of :class:`~mwxml.iteration.page.Page` in the order
        they appear in the XML

:Example:
    .. code-block:: python

        from mwxml import Dump, Page

        # Construct dump file iterator
        dump = Dump.from_file(open("example/dump.xml"))

        # Iterate through pages
        for page in dump.pages:
            # Iterate through a page's revisions
            for revision in p

In [59]:
page_cnt = 0
while page_cnt != 1:
    for page in dump.pages:
        if page.namespace != 0:
            continue
    
        title = page.title
        for revision in page:
            text = revision.text or ""
            print("=== TITLE:", title)
            print(text[:100])
            break
        page_cnt += 1

=== TITLE: Benagil
#REDIRECT [[Carvoeiro#Q2895643]]
=== TITLE: Southern Islands
#REDIRECT [[Singapore/Sentosa and Harbourfront]]
=== TITLE: Rotto
#REDIRECT [[Rottnest Island]]
=== TITLE: Arctic wildlife
{{pagebanner|Wildlife of the Arctic banner.jpg}}
There is no better place than the '''Arctic''' to s
=== TITLE: Zambujeira
#REDIRECT [[Odemira#Q145739]]
=== TITLE: Ouvea
#REDIRECT [[Ouvéa]]
=== TITLE: Gilded Age
#REDIRECT[[Industrialization of the United States]]
=== TITLE: Stockholm metro
#REDIRECT[[Public transport in Stockholm County]]
=== TITLE: Pointe-Fortune
{{pagebanner|}}
[[file:Pointe-Fortune QC.JPG|300px|thumb|right|Bureau municipal et centre communauta
=== TITLE: Mascot
#REDIRECT [[Sydney/Botany Bay]]
=== TITLE: Wildlife of the Arctic
#REDIRECT [[Arctic wildlife]]
=== TITLE: Kelimutu National Park
#REDIRECT [[Kelimutu]]
=== TITLE: Canberra Nature Park
#REDIRECT [[Canberra]]
=== TITLE: Salla National Park
#REDIRECT [[Salla#Q110736317]]
=== TITLE: CR
#REDIRECT [[Costa Rica]]
==

KeyboardInterrupt: 

### DATASET_PAGES_ARTICLES

In [60]:
df = pd.read_xml(
    os.path.join(
        DATA_DIR,
        DATASET_PAGES_ARTICLES,
    )
)

In [61]:
df

,sitename,dbname,base,generator,case,namespaces,title,ns,id,redirect,revision
0,Wikivoyage,enwikivoyage,https://en.wikivoyage.org/wiki/Main_Page,MediaWiki 1.46.0-wmf.4,first-letter,\n,None,NaN,NaN,NaN,None
1,None,None,None,None,None,None,1770,0.0,1.0,NaN,\n
2,None,None,None,None,None,None,1liner,0.0,2.0,NaN,\n
3,None,None,None,None,None,None,2010 FIFA World Cup,0.0,5.0,NaN,\n
4,None,None,None,None,None,None,'s-Gravenhage,0.0,9.0,NaN,\n
...,...,...,...,...,...,...,...,...,...,...,...
78354,None,None,None,None,None,None,Wikivoyage:Temporary account IP viewer/Requests,4.0,214764.0,NaN,\n
78355,None,None,None,None,None,None,Wikivoyage:TAIV,4.0,214765.0,NaN,\n
78356,None,None,None,None,None,None,Wikivoyage:Temporary account IP viewer/TAIV list,4.0,214766.0,NaN,\n
78357,None,None,None,None,None,None,Omeo,0.0,214769.0,NaN,\n


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78359 entries, 0 to 78358
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   sitename    1 non-null      object 
 1   dbname      1 non-null      object 
 2   base        1 non-null      object 
 3   generator   1 non-null      object 
 4   case        1 non-null      object 
 5   namespaces  1 non-null      object 
 6   title       78357 non-null  object 
 7   ns          78358 non-null  float64
 8   id          78358 non-null  float64
 9   redirect    0 non-null      float64
 10  revision    78358 non-null  object 
dtypes: float64(3), object(8)
memory usage: 6.6+ MB


In [63]:
df["title"].sample(50)

50184                             Auroville City
40470                     Category:County Dublin
15260                                    Barmera
22734                             Orontes Valley
74613                                    Nymburk
61988                   Huizhou (disambiguation)
49940                              D-Day Beaches
16170                                      Ladir
30466                              Sweden, Maine
3635                              Boston/Allston
55862                                      Jávea
19492                          Minneapolis/South
16601                              Lassen County
65617                                  Vesilahti
24363                              Poland Spring
66415                             Template:Metre
9721                                   Far hills
53434    Wikivoyage:Votes for deletion/July 2016
57368                           Patiala Division
59879                                   Joynagar
34918               

In [64]:
df["ns"].head()

0    NaN
1    0.0
2    0.0
3    0.0
4    0.0
Name: ns, dtype: float64

In [65]:
df["ns"].value_counts()

ns
0.0      66828
14.0      4862
4.0       2479
6.0       1898
10.0      1880
8.0        213
828.0      185
12.0        13
Name: count, dtype: int64

In [66]:
df["id"].head()

0    NaN
1    1.0
2    2.0
3    5.0
4    9.0
Name: id, dtype: float64

In [69]:
df["revision"].unique()

array([None, '\n      '], dtype=object)

### DATASET_STUB_ARTICLES

In [29]:
df = pd.read_xml(
    os.path.join(
        DATA_DIR,
        DATASET_STUB_ARTICLES,
    )
)

In [30]:
df

,sitename,dbname,base,generator,case,namespaces,title,ns,id,redirect,revision
0,Wikivoyage,enwikivoyage,https://en.wikivoyage.org/wiki/Main_Page,MediaWiki 1.46.0-wmf.4,first-letter,\n,None,NaN,NaN,NaN,None
1,None,None,None,None,None,None,1770,0.0,1.0,NaN,\n
2,None,None,None,None,None,None,1liner,0.0,2.0,NaN,\n
3,None,None,None,None,None,None,2010 FIFA World Cup,0.0,5.0,NaN,\n
4,None,None,None,None,None,None,'s-Gravenhage,0.0,9.0,NaN,\n
...,...,...,...,...,...,...,...,...,...,...,...
78354,None,None,None,None,None,None,Wikivoyage:Temporary account IP viewer/Requests,4.0,214764.0,NaN,\n
78355,None,None,None,None,None,None,Wikivoyage:TAIV,4.0,214765.0,NaN,\n
78356,None,None,None,None,None,None,Wikivoyage:Temporary account IP viewer/TAIV list,4.0,214766.0,NaN,\n
78357,None,None,None,None,None,None,Omeo,0.0,214769.0,NaN,\n
